## This is an analysis of the Medical Cost dataset, by Prabhmeet Singh Deol.

First we will import pandas, and use it to read the dataset.

In [2]:
import pandas as pd

dataSet = pd.read_csv('/Users/deol/Courses/DATA 301/LAB Projects/project-group29/data/raw/Medical_Cost.csv')

In [5]:
## This will give us an overvoiew of the Data.
dataSet.head(10)
## This will tell us the (#Rows,#Columns) of the dataset
dataSet.shape

(1338, 7)

### Now we will try to clean up the data.